|Temperatura °C | B 25/100 | Rt/R25 | Coeficiente de temperatura $\alpha$|
|------------|----------|--------|------------------------------------|
|80.00  | 3492 K | 0.16307 | 2.8 | 
|100.00 | 3492 K | 0.094973 | 2.6 | 


NTC 3950 

|Temperatura °C | B 25/50 | Rt/R25 | Resistencia correspondiente|
|------------|----------|--------|------------------------------------|
|10 | 3950 K | ? | 199.9900 | 
|20 | 3950 K | ? |  125.2450 | 
|30  | 3950 K | ? | 81.0000  | 
|40 | 3950 K | ? | 53.5000  |
|50  | 3950 K | ? | 35.8999 | 
|60 | 3950 K | ? | 25.0000 |
|70  | 3950 K | ? | 17.5500  | 
|80 | 3950 K | ? | 12.5400  |
|90 | 3950 K | ? | 9.1000  |
|100 | 3950 K | ? | 6.7100  |


In [1]:
import math
import numpy as np
import pandas as pd

In [2]:
#Funcion para obtener una diferencia de voltaje de acuerdo a las resistencias de los termistores y la que va en serie con este

#In_voltage es el voltaje con la que se van alimentar las resistencias
#r_25_1 es la resistencia que deberia tener el termistor a la temperatura_1 esto se puede obtener del datasheet del termistor
#resistencia fija es el valor de la resistencia que va en serie con el termistor segun la configuracion del divisor de voltaje

def voltage_divisor_altern_3950(in_voltage, r_25_1,temperatura_1, r_25_2, temperatura_2, resistencia_fija):
    resistencia_temper_1 = r_25_1
    
    #se calcula el divisor de voltaje con el valor de r_25_1
    voltage_divisor_out_1 = in_voltage * (resistencia_temper_1/ (resistencia_temper_1+resistencia_fija)) 
    resistencia_temper_2 = r_25_2
    
    #se calcula el divisor de voltaje con el valor de r_25_2
    voltage_divisor_out_2 = in_voltage * (resistencia_temper_2/ (resistencia_temper_2+resistencia_fija))
    
    #Se calcula la diferencia de voltaje entre ambos divisores, entre mas diferencia halla mejor sera la precision del termistor
    voltage_var = voltage_divisor_out_2 - voltage_divisor_out_1 
    
    #Se imprimen los valores de r_25_1
    print(f'1. Una resistencia de temperatura a {temperatura_1}°C con valor {resistencia_temper_1}\n')
    print(f'El divisor de voltaje es de {voltage_divisor_out_1} V \nTiene una resistencia fija de {resistencia_fija} \n')
    
    #Se imprimen los valores de r_25_2
    print(f'2. Una resistencia de temperatura a {temperatura_2}°C con valor {resistencia_temper_2}\n')
    print(f'El divisor de voltaje es de {voltage_divisor_out_2} V \nTiene una resistencia fija de {resistencia_fija} \n')
    
    
    #Se imprime la diferencia de voltaje
    print(f'\nLa diferencia de voltaje es de : {voltage_var} V entre {temperatura_1}°C y {temperatura_2}°C del termistor\n')

In [6]:
voltage_divisor_altern_3950(3.3, 10000, 25,680,100, 1000)

1. Una resistencia de temperatura a 25°C con valor 10000

El divisor de voltaje es de 2.9999999999999996 V 
Tiene una resistencia fija de 1000 

2. Una resistencia de temperatura a 100°C con valor 680

El divisor de voltaje es de 1.3357142857142856 V 
Tiene una resistencia fija de 1000 


La diferencia de voltaje es de : -1.664285714285714 V entre 25°C y 100°C del termistor



In [3]:
#Esta funcion hace algo similar a la anterior, la diferencia es que busca la resistencia fija ideal para que la 
#diferencia de voltaje sea la maxima
#res_temp_1 es la resistencia que deberia tener el termistor a la temperatura temp_1
#aum es el aumento de valor que se le da a la resistencia fija de manera conveniente, puede ir de 1 en 1, de 10k a 10k 
#lim es el limite de valor a la que llegara la resistencia, esto se pone de acuerdo al analisis que se esta lelvando acabo

def find_fix_resistance(res_temp_1,temp_1,res_temp_2,temp_2,aum,lim):
    
    #voltaje de entrada
    in_voltage = 3.3
    
    #se inicializa la resistencia
    fix_res = 0
    
    #Se crea un array para los valores de la resistencia
    data_fix_res = []
    
    #Se crea un array para los valores de la diferencia de voltaje
    data_diff_voltage = []
    
    #Comienza el ciclo de asignar los valores de la resistencia y seguira hasta que los valores sean iguales o mayores a lim
    while fix_res < lim:
        
        #se agrega el valor de la resistencia fija al array
        data_fix_res.append(fix_res)
        
        #se calcula el divisor de voltaje de acuerdo a las resistencias del termistor a ciertas temperaturas
        voltage_divisor_out_1 = in_voltage * (res_temp_1 / (res_temp_1 + fix_res))
        voltage_divisor_out_2 = in_voltage * (res_temp_2 / (res_temp_2 + fix_res))
        
        #se calcula la diferencia de voltaje
        diff_voltage = voltage_divisor_out_1 - voltage_divisor_out_2
        
        #se aumenta el valor de la resitencia fija de acuerdo al parametro que se coloca
        fix_res += aum
        
        #se agrega el valor de la diferencia de voltaje al array
        data_diff_voltage.append(diff_voltage)
        
    #regresa ambos arrays
    return data_fix_res, data_diff_voltage

#se convierte el a un numpy array
fix_res = np.array(find_fix_resistance(199990, 10,6710,100,10e+3,1e+6))

#se aplica la transpuesta para que las columnas sean la que tiene el mayor numero de registros
fix_res = np.transpose(fix_res)

#se crea un DataFrame y se le asigna el nombte de las columnas
diff_voltage = pd.DataFrame(fix_res,columns = ['Fix Resistance','Voltage Difference'])

#se busca el renglon donde la diferencia de voltaje sea la mayor 
index_max = diff_voltage.index[diff_voltage['Voltage Difference']==diff_voltage['Voltage Difference'].max()].tolist()

#se imprime el renglon que se ha encontrado
print(diff_voltage.iloc[index_max])

   Fix Resistance  Voltage Difference
4         40000.0            2.275924


In [4]:
""" La diferencia entre la primera y la segunda funcion radica que la segunda busca una resistencia ideal, 
pero esta resistencia no necesariamente existe, por ello la primera funcion busca la diferencia de voltaje con una resistencia
especifica y asi se puede usar con las resistencias que existen o son comerciales

"""
# def find_fix_resistance(res_temp_1,temp_1,res_temp_2,temp_2,aum,lim):

fix_res_g = np.array(find_fix_resistance(10000, 25,100,680,100,12e+3))
fix_res_g = np.transpose(fix_res_g)

diff_voltage_g = pd.DataFrame(fix_res_g,columns = ['Fix Resistance','Voltage Difference'])
index_max = diff_voltage_g.index[diff_voltage_g['Voltage Difference']==diff_voltage_g['Voltage Difference'].max()].tolist()
print(diff_voltage_g.iloc[index_max])

    Fix Resistance  Voltage Difference
10          1000.0                 2.7


In [12]:
# def voltage_divisor_altern_3950(in_voltage, r_25_1,temperatura_1, r_25_2, temperatura_2, resistencia_fija):
voltage_divisor_altern_3950(3.3, 10000, 25,940,100, 10000)

Una resistencia de temperatura a 25°C con valor 10000

El divisor de voltaje es de 1.65 V 
Tiene una resistencia fija de 10000 

El divisor de voltaje es de 0.2835466179159049 V 
Tiene una resistencia fija de 10000 

Una resistencia de temperatura a 100°C con valor 940

 La diferencia de voltaje es de : -1.366453382084095


In [13]:
voltage_divisor_altern_3950(3.3, 10000, 25,940,100, 1000)

Una resistencia de temperatura a 25°C con valor 10000

El divisor de voltaje es de 2.9999999999999996 V 
Tiene una resistencia fija de 1000 

El divisor de voltaje es de 1.5989690721649483 V 
Tiene una resistencia fija de 1000 

Una resistencia de temperatura a 100°C con valor 940

 La diferencia de voltaje es de : -1.4010309278350512


In [136]:
voltage_divisor_altern_3950(3.3,12540, 80,896,180, 4.7e+3)

El divisor de voltaje es de 2.4003480278422273 V 
Tiene una resistencia fija de 4700.0 

Una resistencia de temperatura a 80°C con valor 12540

El divisor de voltaje es de 0.5283774124374553 V 
Tiene una resistencia fija de 4700.0 

Una resistencia de temperatura a 180°C con valor 896

 La diferencia de voltaje es de : -1.8719706154047722


In [127]:
voltage_divisor_altern_3950(3.3,12540, 80,896,180, 3900)

El divisor de voltaje es de 1.995274831243973 V 
Tiene una resistencia fija de 8200.0 

Una resistencia de temperatura a 80°C con valor 12540

El divisor de voltaje es de 1.4851106639839033 V 
Tiene una resistencia fija de 8200.0 

Una resistencia de temperatura a 100°C con valor 6710

 La diferencia de voltaje es de : -0.5101641672600696


In [8]:
""" La diferencia entre la primera y la segunda funcion radica que la segunda busca una resistencia ideal, 
pero esta resistencia no necesariamente existe, por ello la primera funcion busca la diferencia de voltaje con una resistencia
especifica y asi se puede usar con las resistencias que existen o son comerciales

"""

fix_res_g = np.array(find_fix_resistance(18216, 10,949.73,100,100,100e+3))
fix_res_g = np.transpose(fix_res_g)

diff_voltage_g = pd.DataFrame(fix_res_g,columns = ['Fix Resistance','Voltage Difference'])
index_max = diff_voltage_g.index[diff_voltage_g['Voltage Difference']==diff_voltage_g['Voltage Difference'].max()].tolist()
print(diff_voltage_g.iloc[index_max])

    Fix Resistance  Voltage Difference
42          4200.0            2.073095


In [12]:
voltage_divisor_altern_3950(3.3,18216, 10,949.73,100, 3900)

Una resistencia de temperatura a 10°C con valor 18216

El divisor de voltaje es de 2.718068366793272 V 
Tiene una resistencia fija de 3900 

El divisor de voltaje es de 0.6462440177082023 V 
Tiene una resistencia fija de 3900 

Una resistencia de temperatura a 100°C con valor 949.73

 La diferencia de voltaje es de : -2.0718243490850696


In [10]:
fix_res_g = np.array(find_fix_resistance(10000, 0,680,100,10,100e+3))
fix_res_g = np.transpose(fix_res_g)

diff_voltage_g = pd.DataFrame(fix_res_g,columns = ['Fix Resistance','Voltage Difference'])
index_max = diff_voltage_g.index[diff_voltage_g['Voltage Difference']==diff_voltage_g['Voltage Difference'].max()].tolist()
print(diff_voltage_g.iloc[index_max])


    Fix Resistance  Voltage Difference
26          2600.0            1.934901


In [8]:
voltage_divisor_altern_3950(3.3,37062, 10,868.37,110, 5000)

Una resistencia de temperatura a 10°C con valor 37062

El divisor de voltaje es de 2.9077219342874803 V 
Tiene una resistencia fija de 5000 

El divisor de voltaje es de 0.4883163467879496 V 
Tiene una resistencia fija de 5000 

Una resistencia de temperatura a 110°C con valor 868.37

 La diferencia de voltaje es de : -2.4194055874995306


In [9]:
voltage_divisor_altern_3950(3.3,37062, 10,868.37,110, 5100)

Una resistencia de temperatura a 10°C con valor 37062

El divisor de voltaje es de 2.900825387789953 V 
Tiene una resistencia fija de 5100 

El divisor de voltaje es de 0.4801346096170311 V 
Tiene una resistencia fija de 5100 

Una resistencia de temperatura a 110°C con valor 868.37

 La diferencia de voltaje es de : -2.4206907781729217


In [9]:
voltage_divisor_altern_3950(3.3,37062, 10,868.37,110, 5600)

Una resistencia de temperatura a 10°C con valor 37062

El divisor de voltaje es de 2.8668276217711313 V 
Tiene una resistencia fija de 5600 

El divisor de voltaje es de 0.44302057550820373 V 
Tiene una resistencia fija de 5600 

Una resistencia de temperatura a 110°C con valor 868.37

 La diferencia de voltaje es de : -2.4238070462629278


In [10]:
voltage_divisor_altern_3950(3.3,37062, 10,868.37,110, 5100)

Una resistencia de temperatura a 10°C con valor 37062

El divisor de voltaje es de 2.900825387789953 V 
Tiene una resistencia fija de 5100 

El divisor de voltaje es de 0.4801346096170311 V 
Tiene una resistencia fija de 5100 

Una resistencia de temperatura a 110°C con valor 868.37

 La diferencia de voltaje es de : -2.4206907781729217


In [ ]:
|